# IBM  Ponder This - July 2024

## Problem Statement

Consider square tiles made of $4\times 4$  squares that can be either black (0) or white (1), such that each tile has an equal number of black and white squares.

We call two tiles equivalent if one can be rotated and/or reflected in order to obtain the other. It is easy to see that a tile can have 1, 2, 4 or 8 equivalent tiles (including itself).

Your goal: Find a tiling of a $16\times 16$ squares board (4 x 4 of such tiles, with 4 x 4 squares in each tile) such that:

1. In each row and column, there are no more than two consecutive squares of the same color.
2. The total number of pairs of adjacent same color squares is minimal. Remember to count both vertical and horizontal pairs.
3. All tiles included in the tiling must be non-equivalent
4. As stated earlier, each tile must have the same number of black and white squares.

Give your solution by first writing the number of pairs from constraint 2, and then the $16\times 16$ grid of squares (no need to describe the individual $4\times 4$ tiles). e.g.

```113``` <br>
```1101001010101001``` <br>
```1010110101010110``` <br>
```0101001010101001``` <br>
```0010110101010110``` <br>
```1101001010101101``` <br>
```0010110101010010``` <br>
```0101100110110100``` <br>
```1010011001001011``` <br>
```0101100100110100``` <br>
```1001001010010010``` <br>
```0010010101101101``` <br>
```1101101101101011``` <br>
```0010010010010010``` <br>
```0011001100110100``` <br>
```1100110101001011``` <br>
```1011011011011011```


A bonus "*" will be given for solving the above problem, but for a $20\times 20$ squares board with the additional constraint that one can only use tiles which have exactly 4 equivalent ones as defined above.

## Solution

We first generate all the possible valid tiles. Then, we select the 16 unique (non-equivalent) tiles containing the lowest number of consecutive squares. Finally, we use backtracking to arrange those tiles (including transformations such as rotation and translation) such that we do not create additional consecutive squares of the same color. For the $16 \times 16$ grid, the answer is found quickly.

In [1]:
from functools import cache
from itertools import combinations
import numpy as np

def generate_valid_tiles():
    valid_tiles = []
    size = 4  # The grid size is 4x4

    # Generate all combinations of 16 squares with exactly 8 being black
    all_combinations = combinations(range(16), 8)

    for comb in all_combinations:
        # Initialize a 4x4 grid with all squares set to white (1)
        grid = np.ones((size, size), dtype=int)

        # Set the positions in the combination to black (0)
        for pos in comb:
            grid[pos // size][pos % size] = 0

        # Check if this configuration is valid
        if is_valid_tile(grid):
            valid_tiles.append(grid)

    return valid_tiles

def is_valid_tile(grid):
    # Check for three consecutive same color horizontally or vertically
    for row in grid:
        if has_three_consecutive(row):
            return False
    for col in grid.T:
        if has_three_consecutive(col):
            return False
    return True

def has_three_consecutive(arr):
    # Check if there are three consecutive 0s or 1s in an array
    for i in range(len(arr) - 2):
        if arr[i] == arr[i+1] == arr[i+2]:
            return True
    return False


def rotate_tile(tile, k=1):
    """Rotate the tile 90 degrees clockwise k times."""
    return np.rot90(tile, -k)

def reflect_tile(tile, axis='horizontal'):
    """Reflect the tile either horizontally or vertically."""
    if axis == 'horizontal':
        return np.fliplr(tile)
    else:
        return np.flipud(tile)

def generate_transformations(tile):
    """Generate all unique transformations of a tile (rotations and reflections)."""
    transformations = set()  # Using a set to avoid duplicates
    # Generate rotations and their reflections
    for k in range(4):
        rotated_tile = rotate_tile(tile, k)
        # Add each rotated version
        transformations.add(tuple(map(tuple, rotated_tile)))
        # Reflect each rotated version horizontally and vertically
        transformations.add(tuple(map(tuple, reflect_tile(rotated_tile, 'horizontal'))))
        transformations.add(tuple(map(tuple, reflect_tile(rotated_tile, 'vertical'))))

    return [np.array(t) for t in transformations]  # Convert each tuple back to an array

def are_equivalent(tile1, tile2):
    """Check if two tiles are equivalent under rotation and reflection."""
    tile2_transforms = generate_transformations(tile2)
    for transformed_tile2 in tile2_transforms:
        if np.array_equal(tile1, transformed_tile2):
            return True
    return False

@cache
def are_equivalent_by_index(tile1, tile2):
    """Check if two tiles are equivalent under rotation and reflection."""
    tile1 = valid_tiles[tile1]
    tile2 = valid_tiles[tile2]
    tile2_transforms = generate_transformations(tile2)
    for transformed_tile2 in tile2_transforms:
        if np.array_equal(tile1, transformed_tile2):
            return True
    return False

def remove_equivalent_tiles(tiles):
    unique_tiles = []  # List to hold one representative from each equivalence class

    for current_tile in tiles:
        # Check if the current tile is equivalent to any tile in the unique list
        if not any(are_equivalent(current_tile, unique_tile) for unique_tile in unique_tiles):
            unique_tiles.append(current_tile)  # Add to unique list if no equivalent found

    return unique_tiles

def count_consecutive_pairs(arr):
    """ Count the number of consecutive identical element pairs in a 1D array """
    pair_count = 0
    current_count = 1
    for i in range(1, len(arr)):
        if arr[i] == arr[i - 1]:
            current_count += 1
        else:
            if current_count > 1:
                pair_count += 1
            current_count = 1
    # Check the last sequence
    if current_count > 1:
        pair_count += 1
    return pair_count

def consecutive_pairs_tile(tile):
    """ Calculate the number of consecutive pairs in both rows and columns of a 2D array """
    total_pairs = 0
    # Check rows
    for row in tile:
        total_pairs += count_consecutive_pairs(row)
    # Check columns
    for col in tile.T:  # Transpose to get columns
        total_pairs += count_consecutive_pairs(col)
    return total_pairs

In [2]:
# Generate the tiles
valid_tiles = generate_valid_tiles()
print("Number of valid tiles:", len(valid_tiles))

# Assuming 'valid_tiles' is a list of numpy arrays as generated earlier
unique_tiles = remove_equivalent_tiles(valid_tiles)
print("Number of unique tiles:", len(unique_tiles))

# Sort tiles by the number of consecutive pairs
valid_tiles = sorted(valid_tiles, key=consecutive_pairs_tile)

Number of valid tiles: 882
Number of unique tiles: 130


In [3]:
selected = set()
for i in range(len(valid_tiles) - 1):
    eq = False
    for s in selected:
        if are_equivalent_by_index(i, s):
            eq = True
            break
    if not eq:
        selected.add(i)
    if len(selected) == 16:
        break

tiles = [valid_tiles[x] for x in selected]

In [4]:
for tile in tiles:
    print(consecutive_pairs_tile(tile))

0
5
3
5
3
6
4
4
5
4
5
5
5
4
4
4


In [5]:
def check_horizontal_pairing(left_matrix, right_matrix):
    """ Check if two matrices can be horizontally paired without extending consecutive sequences. """
    return not np.any(left_matrix[:, -1] == right_matrix[:, 0])

def check_vertical_pairing(top_matrix, bottom_matrix):
    """ Check if two matrices can be vertically paired without extending consecutive sequences. """
    return not np.any(top_matrix[-1, :] == bottom_matrix[0, :])

grid = [[0] * 4 for _ in range(4)]

def solve(i, j, tiles, visited):
    if i == 4 and j == 0:
        return grid.copy()
    for k, tile in enumerate(tiles):
        if k not in visited:
            transformations = generate_transformations(tile)
            for t in transformations:
                if i > 0 and not check_vertical_pairing(grid[i - 1][j], t):
                    continue
                if j > 0 and not check_horizontal_pairing(grid[i][j - 1], t):
                    continue
                grid[i][j] = t
                visited.add(k)
                if j < 3:
                    res = solve(i, j+1, tiles, visited)
                else:
                    res = solve(i+1, 0, tiles, visited)
                if res:
                    return res
                grid[i][j] = 0
                visited.remove(k)
    return

In [6]:
visited = set()
grid = solve(0, 0, tiles, visited)

final_grid = np.vstack((
    np.hstack(grid[0]),
    np.hstack(grid[1]),
    np.hstack(grid[2]),
    np.hstack(grid[3]),
))

In [7]:
visited

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [8]:
final_grid

array([[0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1],
       [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1],
       [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]])

In [9]:
total_consecutives = 0
for tile in tiles:
    total_consecutives += consecutive_pairs_tile(tile)

In [10]:
print(total_consecutives)
for i in range(len(final_grid)):
    print("".join(final_grid[i].astype(str)))

66
0101011010101001
1010100101010010
0101010010101101
1010101101101010
0101010010010101
1010110101101011
0100101010010100
1011010101010011
0100101010101100
1011010101101010
0100100101010101
1011011010101010
0100100101010101
1011011010101010
1010100101010110
0101011011001001
